## Connection Set up


### overview
- Detection thread: captures frames and runs them against our SORT algorithm then sends tracks and video frames in a queue.Queue → no socket calls.

- Prediction thread: reads from queue → processes using Kalman Filter annotates frames with where object is predicted to go along with other track data  → emits via socketio.emit to HMI/frontend.

<b> ok we are experimenting the following connection </b>


In [1]:

import cv2, base64 # type: ignore
import threading
import time
import sys
from flask import Flask, render_template, jsonify, request
from flask_socketio import SocketIO


app = Flask(
    __name__,
    template_folder='../Human Machine Interface/templates',
    static_folder='../Human Machine Interface/static'
)

hmi_socket = SocketIO(app, cors_allowed_origins="*") 


# Shared state placeholder for future video frames
shared_state = {
    "running": True,
    "last_frame_ts": None,
}



def video_thread():
    cap = cv2.VideoCapture(0)  # video source
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        # encode frame as JPEG and base64
        _, buffer = cv2.imencode('.jpg', frame)
        jpg_as_text = base64.b64encode(buffer).decode('utf-8')
        # this is the actual emission
        hmi_socket.emit('video_frame', {'image': jpg_as_text})
        time.sleep(0.03)  # ~30 FPS


@app.route('/')
def index():
    # Render front-end. The live feed area will show placeholder text.
    return render_template('index.html')


@app.route('/neutralize', methods=['POST'])
def neutralize():
    # This is where we will add the neutralize command. For now we print.
    print('neutralize', file=sys.stdout)
    sys.stdout.flush()
    return jsonify({"status": "ok", "action": "neutralize"})

def run_flask():
    # Note: use debug=False in production
    hmi_socket.run(app, host='0.0.0.0', port=5000, debug=True, use_reloader=False, allow_unsafe_werkzeug=True)



if __name__ == '__main__':
    # Start background video thread (daemon so it exits with main process)
    detection_engine_thread = threading.Thread(target=video_thread, daemon=True)
    detection_engine_thread.start()

    # Start Flask in its own thread
    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Shutting Down Uzziah...")


Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.20.10.2:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Nov/2025 17:42:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2025 17:42:27] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2025 17:42:27] "GET /static/script.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2025 17:42:27] "GET /socket.io/?EIO=4&transport=polling&t=PfFYAWb HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2025 17:42:27] "GET /socket.io/?EIO=4&transport=polling&t=PfFYAWd HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2025 17:42:28] "POST /socket.io/?EIO=4&transport=polling&t=PfFYAsn&sid=eHJu6m3B3zbIHTgCAAAA HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2025 17:42:28] "POST /socket.io/?EIO=4&transport=polling&t=PfFYAsq&sid=hSugL7k9P4YFDdYcAAAB HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2025 17:42:28] "GET /socket.io/?EIO=4&transport=polling&t=PfFYAsp&sid=eHJu6m3B3zbIHTgCAAAA HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2025 17:42:28] "GET /socket.io/?EIO=

Shutting Down Uzziah...


<b>The Connection Above Streams from camera in a separate thread detection_engine_thread (via video thread) to the front end (flask thread) in a different thread via hmi socket</b>